In [28]:
import boto3
import datetime
from imdb import IMDb

In [29]:
ia = IMDb()
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('plex-select-collection')
bucket_name = 'manny-and-meche-plex'
object_summary_iterator = s3_resource.Bucket(bucket_name).objects.all()

In [30]:
def update_dynamodb_movie_list():
    already_added_movies = []
    for i in table.scan()['Items']:
        already_added_movies.append(i['Search Title'])
    return already_added_movies

In [31]:
already_updated_movies = update_dynamodb_movie_list()
for i in object_summary_iterator:
    key = i.key
    if ('Movies/' in key) and \
       (key!='Movies/'):
        metadata = s3_client.head_object(Bucket=bucket_name, Key=key)['Metadata']
        movie_proper_title = metadata['proper_movie_title']
        today = datetime.datetime.today().strftime('%Y %m %d')
        m_search = movie_proper_title.replace(" ", "").replace(":", "").replace("-", "").replace("'", "").replace(".", "").replace(",","")
        m_search = m_search.lower()
        if (m_search not in already_updated_movies):
            movie = ia.get_movie(metadata['movieid'])
            data_input = {}
            data_input['Search Title'] = m_search
            data_input['Title'] = movie_proper_title
            data_input['Plot'] = movie['plot'][0]
            data_input['Genres'] = movie['genres']
            release_date = movie['original air date'][:11]
            datetime_object = datetime.datetime.strptime(release_date, '%d %b %Y')
            data_input['Release Date'] = datetime_object.strftime('%Y %m %d')
            data_input['Add Date'] = today
            data_input['Year'] = datetime_object.strftime('%Y')
            table.put_item(Item=data_input)
            print('Added '+ movie_proper_title)
        else:
            print('Already have ' + movie_proper_title)

Already have 10 Cloverfield Lane
Already have 13 Hours
Already have 28 Days Later
Already have 300
Already have A Beautiful Mind
Already have A Christmas Carol
Already have A Quiet Place
Already have Alpha
Already have Amadeus
Already have Amelie
Already have American Hustle
Already have Anchorman: The Legend of Ron Burgundy
Already have Angels and Demons
Already have Ant-Man and the Wasp
Already have Ant-Man
Already have Avatar
Already have Avengers: Age of Ultron
Already have Avengers: Infinity War
Already have Baby Driver
Already have Batman Begins
Already have Batman v Superman: Dawn of Justice
Already have Beauty and the Beast
Already have Big Hero 6
Already have Birdman
Already have Black Panther
Already have Blade Runner
Already have Borat
Already have Brave
Already have Braveheart
Already have Bruno
Already have Captain America: Civil War
Already have Captain America: The First Avenger
Already have Captain America: The Winter Soldier
Already have Casablanca
Already have Casino 

Added X-Men: First Class
Added Zombieland
